In [47]:
import tqdm
from tqdm import tqdm
import pandas as pd 
import random
import json
import spacy
import translators as ts
import scipy
import pickle
import numpy as np

df = pd.read_excel('../Excel_files/Full_Poem_Dataset_12-17.xlsx')
df = df.drop(columns=['Unnamed: 0'])
records = df.to_dict('records')
random.choice(records)

{'Text': 'Перебивали речь, кивали в такт, брались за струны, выходили вон,\nменялись, оставались на местах — и вечер наступал со всех сторон.\n \nДо встречи в среду, ливеньливень рос, промокнешь навсегда, постой идти. \nДень уплывал как спасшийся матрос — тогда писал я, господи прости\n \nИ так как ты была там редкий гость, и олицетворяла свет извне,\nи рядом с этим днем стоял другой, казалось все осмысленным вполне.\nИ открываясь, хлопало окно, которое никто не закрывал,\nпока один — из старого кино — одну смешную песенку играл.\n \nИ смерть, не улыбаясь никому, вставала, выходила в темноту,\nСмеясь, слова ловили на лету, кружась передавали их в дыму.\n \nВставали, уходили без зонтов, и на прощанье кто-то говорил —\n«до встречи» — и не спрашивал никто, куда они — у тех, кто уходил.\n \nИ ливень на незаданный вопрос, не отвечал на разные лады.\nДень уплывал, кончался, как матрос на плотике без пищи и воды.\n \nЧерез плечо смотрящий мне в тетрадь: А в чем сюжет, поэт, теряю нить.\n— Я з

In [48]:
df['Before or after'].value_counts()

Before    1661
After     1561
Name: Before or after, dtype: int64

In [49]:
# average lines
numLines = []
theseRecs = []
for rec in records:
    # lines = rec['Text'].split('\n')
    lines = [t for t in rec['Text'].split('\n') if t.replace('*','').strip() not in ['','\t',' ']]
    if len(lines) in [2,3]:
        theseRecs.append(rec)
    numLines.append(len(lines))

In [50]:
# how many authors
authorSet = set()
for d in records:
    authorSet.add(d['Author'])
len(authorSet)

677

In [51]:
authorsDf = pd.read_json('../notebooks/Thesis_Authors16.json')
authorsDf = authorsDf.fillna('N/A')
aRecs = authorsDf.to_dict('records')
authorDict = dict()
for a in aRecs:
    authorDict[a['Author']] = a
authorsDf.head()

,Number of Poems,Author,City,Country,Identification,Notes,Link to bio
0,3,Виген Аракелян,Razdan,Armenia,N/A,N/A,https://polutona.ru/?show=arakelian
1,1,Герман Лукомников,Baku,Azerbaijan,N/A,N/A,https://ru.wikipedia.org/wiki/%D0%9B%D1%83%D0%...
2,1,Александр Иличевский,Sumgait,Azerbaijan,N/A,N/A,https://ru.wikipedia.org/wiki/%D0%98%D0%BB%D0%...
3,7,Леонид Шваб,Babruysk,Belarus,N/A,N/A,http://www.litkarta.ru/world/israel/persons/sh...
4,6,Ирина Валерина,Babruysk,Belarus,N/A,N/A,N/A


In [52]:
# places = set()
# for rec in aRecs:
#     if rec['Author'] in authorSet:
#         places.add(rec['City']+', '+rec['Country'])
#     else:
#         print(rec)
# for p in sorted(places):
#     print(p)

### Statistics for Methods section

In [53]:
nlp = spacy.load("ru_core_news_lg")

In [54]:
for rec in tqdm(records):
    doc = nlp(rec['Text'])
    rec['doc'] = doc

100%|██████████████████████████████| 3222/3222 [02:34<00:00, 20.91it/s]


In [13]:
# look at lemma counts
tokenCt = dict()
poemCt = dict()
tokenLemmas = dict()

for p in ['Before','After']:
    tokenCt.setdefault(p, 0)
    poemCt.setdefault(p, 0)
    tokenLemmas.setdefault(p, set())
for rec in tqdm(records):
    poemCt[rec['Before or after']] += 1

    for t in rec['doc']:
        tokenCt[rec['Before or after']] += 1 
        tokenLemmas[p].add(t.lemma_)
        
print(tokenCt)
print(poemCt)
print(len(tokenLemmas['Before']),len(tokenLemmas['After']))

100%|█████████████████████████████████████| 3222/3222 [00:00<00:00, 4979.36it/s]

{'Before': 291769, 'After': 266511}
{'Before': 1661, 'After': 1561}
0 43565


### Little experiments

In [12]:
tokenLemmas['Before']

set()

In [104]:
### Tense breakdown each time period
tenseDict = {'Before' : dict(), 'After' : dict()}
for p in tenseDict:
    for t in ['Past','Pres','Fut']:
        tenseDict[p][t] = 0
for r in tqdm(records):
    p = r['Before or after']
    for t in r['doc']:
        if t.pos_ == 'VERB' and 'Tense' in t.morph.to_dict():
            tense = t.morph.to_dict()['Tense']
            m = t.morph
            tenseDict[p].setdefault(tense, 0)
            tenseDict[p][tense]+=1
for p in tenseDict:
    print(p)
    for t in tenseDict[p]:
        print(t,tenseDict[p][t]/(tenseDict[p]['Past']+tenseDict[p]['Pres']+tenseDict[p]['Fut']))
    print()

100%|█████████████████████████████████████| 3222/3222 [00:01<00:00, 3187.59it/s]


In [113]:
a1 = tenseDict['Before']['Pres']*[1]+ tenseDict["Before"]['Past']*[0] + tenseDict['Before']['Fut']*[0]
a2 = tenseDict['After']['Pres']*[1]+ tenseDict["After"]['Past']*[0] + tenseDict['After']['Fut']*[0]
test = scipy.stats.ttest_ind(a1, a2, axis=0, equal_var=True, nan_policy='propagate', 
                          permutations=None, random_state=None, alternative='two-sided', trim=0)
print(test)

Ttest_indResult(statistic=6.0487816625211215, pvalue=1.4693138352414718e-09)


In [114]:
# fut : Ttest_indResult(statistic=-7.359306571050845, pvalue=1.8756963438929145e-13)
# Pres: Ttest_indResult(statistic=6.0487816625211215, pvalue=1.4693138352414718e-09)

In [ ]:
posDict['Before

In [147]:
### Tense breakdown each time period
posDict = {'Before' : dict(), 'After' : dict()}
totalDict = {'Before' : 0, 'After' : 0}
adjs = set()

for r in tqdm(records):
    p = r['Before or after']
    for t in r['doc']:
        if t.pos_:
            pos = t.pos_
            posDict[p].setdefault(pos, 0)
            posDict[p][pos]+=1
        if t.pos_ == 'ADJ':
            adjs.add(t.lemma_)
        if t.pos_ in ['NOUN', 'ADP', 'PROPN', 'ADJ', 'DET', 'SCONJ', 'VERB', 'ADV', 'PRON', 'CCONJ', 'PART', 'AUX', 'NUM', 'INTJ']:
            totalDict[p]+=1

100%|█████████████████████████████████████| 3222/3222 [00:02<00:00, 1462.79it/s]


In [140]:
(posDict['After']['ADJ']/totalDict['After']-posDict['Before']['ADJ']/totalDict['Before'])/(posDict['Before']['ADJ']/totalDict['Before'])

-0.06345136188070334

In [141]:
for t in posDict['Before']:
    print(p)
    for p in ['Before','After']:
        print(t,posDict[p][t]/totalDict[p])
    print()

Before
NOUN 0.28718523363407267
NOUN 0.2867860716751128

After
ADP 0.1053998422561222
ADP 0.10570928608505333

After
PROPN 0.03020699074697497
PROPN 0.038443632238282024

After
ADJ 0.09392974626320143
ADJ 0.08796977594169239

After
SPACE 0.1961747109631995
SPACE 0.2013849125867577

After
DET 0.031895042610084065
DET 0.0323806262073837

After
PUNCT 0.2014697112517554
PUNCT 0.2213987189023444

After
SCONJ 0.026551949675855568
SCONJ 0.023898839306259398

After
VERB 0.1590183329165304
VERB 0.16362624484269897

After
ADV 0.06487216974780216
ADV 0.06370704924787687

After
PRON 0.08523940519015832
PRON 0.08046192293079182

After
CCONJ 0.055017986649481564
CCONJ 0.053395122838749726

After
PART 0.04244656907065771
PART 0.04711806540340663

After
AUX 0.007401458169016794
AUX 0.007598824857789599

After
NUM 0.010402439258988516
NUM 0.00852459718201326

After
INTJ 0.0004328338110536137
INTJ 0.0003799412428894799

After
X 0.00498239809168382
X 0.003242878777338378

After
SYM 0.0005674932189369601


In [144]:
for pos in ['NOUN', 'ADP', 'PROPN', 'ADJ', 'DET', 'SCONJ', 'VERB', 'ADV', 'PRON', 'CCONJ', 'PART', 'AUX', 'NUM', 'INTJ']:
    a1 = posDict['Before'][pos]*[1]+ totalDict["Before"]*[0]
    a2 = posDict['After'][pos]*[1]+ totalDict["After"]*[0]
    test = scipy.stats.ttest_ind(a1, a2, axis=0, equal_var=True, nan_policy='propagate', 
                              permutations=None, random_state=None, alternative='two-sided', trim=0)
    if test.pvalue < .05:
        print(pos, test)

PROPN Ttest_indResult(statistic=-13.759636313472408, pvalue=4.557583688404265e-43)
ADJ Ttest_indResult(statistic=5.93134393655854, pvalue=3.0069309691234293e-09)
SCONJ Ttest_indResult(statistic=5.168783539064797, pvalue=2.35734161794761e-07)
VERB Ttest_indResult(statistic=-3.340947962508741, pvalue=0.0008349953358985565)
PRON Ttest_indResult(statistic=5.000492327496617, pvalue=5.720666020345868e-07)
CCONJ Ttest_indResult(statistic=2.1289808190481043, pvalue=0.033256432733142285)
PART Ttest_indResult(statistic=-6.784762061190034, pvalue=1.1643386677297055e-11)
NUM Ttest_indResult(statistic=6.011905477380559, pvalue=1.8351374845103045e-09)


### Calculating two sample one sided t-test

In [55]:
textDict = dict()
lemmaCts = dict()
totalPoemCts = dict()
allLemmas = set()
scipyArray = dict()
bigData = dict()
for r in tqdm(records):
    for t in r['doc']:
        lemmaCts.setdefault(t.lemma_, {'Before' : 0, 'After' : 0, 'nlpToken' : t})
        textDict.setdefault(t.lemma_, {'Before' : [], 'After' : []})
        scipyArray.setdefault(t.lemma_, {'Before' : [], 'After' : []})
        allLemmas.add(t.lemma_)
        
for aff in ['Russia','Non-Russia']:
    print(aff)
    for r in tqdm(records):
        if r['Author'] not in authorDict:
            continue
        if authorDict[r['Author']]['Country'] == 'Russia':
            thisaff = 'Russia'
        elif authorDict[r['Author']]['Country'] == 'N/A':
            continue
        else:
            thisaff = 'Non-Russia'
        if thisaff != aff:
            continue
            
        p = r['Before or after']
        totalPoemCts.setdefault(p, 0)
        lemmasFound = set()
        totalPoemCts[p] += 1
        for t in r['doc']:
            if t.lemma_ not in lemmasFound:
                # lemmaCts.setdefault(t.lemma_, {'Before' : 0, 'After' : 0})
                lemmaCts[t.lemma_][p] += 1
                textDict[t.lemma_][p].append({'ID' : r['UniqueIndex'], 'tokenText' : t.text})
                lemmasFound.add(t.lemma_)
                
            else:
                continue
        # create arrays for scipy
        recLemmas = set([t.lemma_ for t in r['doc']])
        for rl in recLemmas:
            scipyArray[rl][p].append(1)
        for lem in allLemmas:
            if lem not in recLemmas:
                scipyArray[lem][p].append(0)
                
    lemmaFreqs = dict()
    for word in allLemmas:
        lemmaFreqs.setdefault(word, {'Before' : 0, 'After' : 0})
        for p in ['Before','After']:
            lemmaFreqs[word][p] = lemmaCts[word][p]/totalPoemCts[p]
            lemmaFreqs[word][p+'Ct'] = lemmaCts[word][p]
            lemmaFreqs[word]['nlp'] = lemmaCts[word]['nlpToken']

    jsonData = []
    theselemmas = set()
    for keyword in tqdm(allLemmas):
        if lemmaCts[keyword]['Before'] + lemmaCts[keyword]['After'] >= 25:
            theselemmas.add(keyword)
            a1 = scipyArray[keyword]['Before']
            a2 = scipyArray[keyword]['After']
            test = scipy.stats.ttest_ind(a1, a2, axis=0, equal_var=True, nan_policy='propagate', 
                                  permutations=None, random_state=None, alternative='two-sided', trim=0)
            lemmaFreqs[keyword]['stats'] = {'statistic' : test.statistic, 'pvalue' : test.pvalue}
            pos = lemmaFreqs[keyword]['nlp'].pos_
            del lemmaFreqs[keyword]['nlp']
            jsonData.append({
                'keyword' : keyword,
                'pos' : pos,
                'info' : lemmaFreqs[keyword]
            })
    print(len(jsonData))
    bigData[aff] = jsonData

100%|█████████████████████████████| 3222/3222 [00:04<00:00, 679.06it/s]


Russia


100%|██████████████████████████| 43565/43565 [00:10<00:00, 3973.98it/s]


923
Non-Russia


100%|█████████████████████████| 43565/43565 [00:00<00:00, 68916.70it/s]

1412


In [65]:
random.choice(bigData['Russia'])

{'keyword': 'назад',
 'pos': 'ADV',
 'info': {'Before': 0.031818181818181815,
  'After': 0.038834951456310676,
  'BeforeCt': 35,
  'AfterCt': 28,
  'stats': {'statistic': -0.8009665987099532, 'pvalue': 0.42325558830164767}}}

In [64]:
with open(f'../sigFreqTables/frequencyData3-26.json', 'w', encoding='utf-8') as f:
    json.dump(bigData, f, ensure_ascii=False, indent=4)

### Diff between Rus/Non-Rus?

In [113]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

In [114]:
random.choice(bigData['Russia'])

{'keyword': 'них',
 'pos': 'PRON',
 'info': {'Before': 0.06909090909090909,
  'After': 0.0651872399445215,
  'BeforeCt': 76,
  'AfterCt': 47,
  'stats': {'statistic': 0.3244475462185382, 'pvalue': 0.7456365332324566}}}

In [121]:
ctDict = dict()
for aff in ['Russia','Non-Russia']:
    
    print(aff)
    for r in tqdm(records):
        p = r['Before or after']
        if r['Author'] not in authorDict:
            continue
        if authorDict[r['Author']]['Country'] == 'Russia':
            thisaff = 'Russia'
        elif authorDict[r['Author']]['Country'] == 'N/A':
            continue
        else:
            thisaff = 'Non-Russia'

        if thisaff != aff:
            continue
            
        ctDict.setdefault(aff, {'Before' : 0,'After' : 0})
        ctDict[aff][p]+=1

Russia


100%|██████████████████████████| 3222/3222 [00:00<00:00, 328337.61it/s]


Non-Russia


100%|██████████████████████████| 3222/3222 [00:00<00:00, 911264.16it/s]


In [122]:
ctDict

{'Russia': {'Before': 1100, 'After': 721},
 'Non-Russia': {'Before': 405, 'After': 550}}

In [127]:
diffBtwn = []
for d in tqdm(bigData['Russia']):
    for e in bigData['Non-Russia']:
        if d['keyword'] == e['keyword']:
            # Russian group proportions and sample sizes
            russian_before = d['info']['Before']  # Example: 15% of words in the "Before" period
            russian_after = d['info']['After']   # Example: 25% of words in the "After" period
            n_russian_before = 1100  # Example: 1000 words in the Russian group before
            n_russian_after = 721   # Example: 1000 words in the Russian group after

            # Non-Russian group proportions and sample sizes
            non_russian_before = e['info']['Before']  # Example: 12% of words in the "Before" period
            non_russian_after = e['info']['After']  # Example: 22% of words in the "After" period
            n_non_russian_before = 405  # Example: 1000 words in the non-Russian group before
            n_non_russian_after = 550   # Example: 1000 words in the non-Russian group after
            
            russian_diff = russian_after - russian_before
            non_russian_diff = non_russian_after - non_russian_before
            
            pooled_prop = ((russian_before * n_russian_before + non_russian_before * n_non_russian_before) +
              (russian_after * n_russian_after + non_russian_after * n_non_russian_after)) / \
              (n_russian_before + n_russian_after + n_non_russian_before + n_non_russian_after)
            
            
            count = np.array([russian_after * n_russian_after - russian_before * n_russian_before,
                              non_russian_after * n_non_russian_after - non_russian_before * n_non_russian_before])
            nobs = np.array([n_russian_before + n_russian_after, n_non_russian_before + n_non_russian_after])

            z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

            alpha = 0.05  # Example: significance level of 0.05
            if p_value < alpha:
                diffBtwn.append({'Russia' : d, 'Non-Russia' : e, 'p_value' : p_value})
                # print("The difference in frequency changes between the Russian and non-Russian groups is statistically significant.")
            else:
                # print("The difference in frequency changes between the Russian and non-Russian groups is not statistically significant.")
                continue
len(diffBtwn)

100%|██████████████████████████████| 923/923 [00:00<00:00, 2546.22it/s]


192

In [128]:
# in the notaligned, are there any that go in opposite directions?
oppDirs = []
for pair in diffBtwn:
    dirR = pair['Russia']['info']['After'] - pair['Russia']['info']['Before']
    dirNR = pair['Non-Russia']['info']['After'] - pair['Non-Russia']['info']['Before']
    if np.sign(dirR) != np.sign(dirNR):
        oppDirs.append(pair)
len(oppDirs)

4

In [129]:
oppDirs

[{'Russia': {'keyword': 'ужас',
   'pos': 'NOUN',
   'info': {'Before': 0.02,
    'After': 0.019417475728155338,
    'BeforeCt': 22,
    'AfterCt': 14,
    'stats': {'statistic': 0.08728204215536717,
     'pvalue': 0.9304569347396332}}},
  'Non-Russia': {'keyword': 'ужас',
   'pos': 'NOUN',
   'info': {'Before': 0.017940199335548173,
    'After': 0.02989771833202203,
    'BeforeCt': 27,
    'AfterCt': 38,
    'stats': {'statistic': -2.076592080070808, 'pvalue': 0.0379310826006974}}},
  'p_value': 5.980813705380458e-65},
 {'Russia': {'keyword': 'число',
   'pos': 'NOUN',
   'info': {'Before': 0.014545454545454545,
    'After': 0.020804438280166437,
    'BeforeCt': 16,
    'AfterCt': 15,
    'stats': {'statistic': -1.0094815891374527,
     'pvalue': 0.31287798753851914}}},
  'Non-Russia': {'keyword': 'число',
   'pos': 'NOUN',
   'info': {'Before': 0.013953488372093023,
    'After': 0.013375295043273014,
    'BeforeCt': 21,
    'AfterCt': 17,
    'stats': {'statistic': 0.1305750330807814

In [124]:
random.choice(diffBtwn)

{'Russia': {'keyword': 'молчать',
  'pos': 'VERB',
  'info': {'Before': 0.034545454545454546,
   'After': 0.05131761442441054,
   'BeforeCt': 38,
   'AfterCt': 37,
   'stats': {'statistic': -1.7619226647406203, 'pvalue': 0.07825031630214}}},
 'non-Russia': {'keyword': 'молчать',
  'pos': 'VERB',
  'info': {'Before': 0.03122923588039867,
   'After': 0.043273013375295044,
   'BeforeCt': 47,
   'AfterCt': 55,
   'stats': {'statistic': -1.6807271068920744,
    'pvalue': 0.09292851473192604}}},
 'p_value': 3.9829236795424123e-07}

In [75]:
# find the words which changed for one but not the other
notaligned = []
alignedSig = []
alignedNotSig = []
for d in tqdm(bigData['Russia']):
    for e in bigData['Non-Russia']:
        if d['keyword'] == e['keyword']:
            if d['info']['stats']['pvalue'] < .05 and e['info']['stats']['pvalue'] > .05:
                notaligned.append({'Russia':d, 'Non-Russia':e})
            elif d['info']['stats']['pvalue'] > .05 and e['info']['stats']['pvalue'] < .05:
                notaligned.append({'Russia':d, 'Non-Russia':e})
            elif d['info']['stats']['pvalue'] < .05 and e['info']['stats']['pvalue'] < .05:
                alignedSig.append({'Russia':d, 'Non-Russia':e})
            else:
                alignedNotSig.append({'Russia':d, 'Non-Russia':e})

100%|██████████████████████████████| 923/923 [00:00<00:00, 4556.54it/s]


In [76]:
len(notaligned), len(alignedSig), len(alignedNotSig)

(139, 204, 580)

In [132]:
words =['язык', 'речь', 'слово', 'словарь', 'говорить', 'сказать',
                    'значение', 'смысл', 'разговор','тишина','молчать','замолчать','погорвоить','сказывать',
                'беседа','беседовать','произношение','произносить','тема','значение','значить']
for a in notaligned:
    if a['Russia']['keyword'] == 'любить':
        print(a)

In [105]:
# in the notaligned, are there any that go in opposite directions?
oppDirs = []
for pair in notaligned:
    dirR = pair['Russia']['info']['After'] - pair['Russia']['info']['Before']
    dirNR = pair['Non-Russia']['info']['After'] - pair['Non-Russia']['info']['Before']
    if np.sign(dirR) != np.sign(dirNR):
        oppDirs.append(pair)
len(oppDirs)

2

In [106]:
oppDirs

[{'Russia': {'keyword': 'ужас',
   'pos': 'NOUN',
   'info': {'Before': 0.02,
    'After': 0.019417475728155338,
    'BeforeCt': 22,
    'AfterCt': 14,
    'stats': {'statistic': 0.08728204215536717,
     'pvalue': 0.9304569347396332}}},
  'Non-Russia': {'keyword': 'ужас',
   'pos': 'NOUN',
   'info': {'Before': 0.017940199335548173,
    'After': 0.02989771833202203,
    'BeforeCt': 27,
    'AfterCt': 38,
    'stats': {'statistic': -2.076592080070808,
     'pvalue': 0.0379310826006974}}}},
 {'Russia': {'keyword': 'пока',
   'pos': 'ADV',
   'info': {'Before': 0.11545454545454545,
    'After': 0.08599167822468794,
    'BeforeCt': 127,
    'AfterCt': 62,
    'stats': {'statistic': 2.0172028367794943,
     'pvalue': 0.04382099141501074}}},
  'Non-Russia': {'keyword': 'пока',
   'pos': 'ADV',
   'info': {'Before': 0.11029900332225914,
    'After': 0.1109362706530291,
    'BeforeCt': 166,
    'AfterCt': 141,
    'stats': {'statistic': -0.053319540563253234,
     'pvalue': 0.9574811505631675

In [111]:
interesting = []
for r in records:
    docLemmas = set([t.lemma_ for t in r['doc']])
    if 'язык' in docLemmas and 'ужас' in docLemmas:
        interesting.append(r)
len(interesting)

4

In [112]:
[i['UniqueIndex'] for i in interesting]

[1523, 1273, 523, 1174]

#### Significant word breakdown

In [18]:
# filter data
data = [d for d in jsonData if not d['info']['Before']*d['info']['After']==0 ]
data = [d for d in data if d['info']['stats']['pvalue'] < .05]
data = [d for d in data if d['keyword'].isalpha()]
irrelevantWords = ['авторский', 'блог', 'личный', 'источник','автор']
data = [d for d in data if d['keyword'] not in irrelevantWords]
print(len(jsonData), len(data))

1626 421


In [19]:
lemsForPCA = [d['keyword'] for d in data]
len(lemsForPCA)

421

In [20]:
# with open('word4map.txt', 'w') as f:
#     f.write('\n'.join(lemsForPCA))

In [149]:
posDict = {'increase' : dict(), 'decrease' : dict()}
for d in data:
    which = ''
    if d['info']['After'] - d['info']['Before'] > 0:
        which = 'increase'
    else:
        which = 'decrease'
    pos = d['info']['nlp'].pos_
    posDict[which].setdefault(pos, [])
    posDict[which][pos].append(d)

In [22]:
sorted([p for p in set(list(posDict['increase'].keys()) + list(posDict['decrease'].keys()))])

['ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'SCONJ',
 'VERB']

In [155]:
random.sample((adjs - iodAdjs),5)

/var/folders/jj/_szc94p56q91q_7c209d1d9w0000gn/T/ipykernel_1438/3898841191.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample((adjs - iodAdjs),5)


['скушно', 'крутой', 'цветущий', 'обычный', 'отъявленный']

In [150]:
iodAdjs = set()
for iod in ['increase','decrease']:
    for pos in posDict[iod]:
        if pos == 'ADJ':
            print(pos,len(sorted(posDict[iod][pos], key=lambda x:x['keyword'])))
            for n in sorted(posDict[iod][pos], key=lambda x:x['keyword']):
                print(n['keyword'])
                iodAdjs.add(n['keyword'])
            print()

ADJ 17
безумный
божий
весенний
вечный
виноватый
военный
злой
кровавый
мирный
мировой
родный
российский
русский
святой
страшный
чужой
ядерный

ADJ 45
белый
высоко
глубокий
голубой
горячий
дикий
длинный
долгий
другой
жаркий
зелёный
красивый
круглый
ледяной
летний
лёгкий
медленный
милый
мокрый
молодой
мягкий
небольшой
невидимый
нежный
непонятный
ночной
осенний
полный
прозрачный
пустой
светлый
слабый
сложный
случайный
смешной
собачий
сплошной
стальной
стеклянный
странный
сырой
тёмный
тёплый
узкий
школьный



## Similarity

In [42]:
import time
import os.path

from django.conf import settings

from navec import Navec
from numpy import dot
from numpy.linalg import norm
from annoy import AnnoyIndex


NAVEC_PATH = "data4notebooks/navec_hudlit_v1_12B_500K_300d_100q.tar"
# ANNOY_INDEX_PATH = os.path.join(settings.ROOT_DIR, "parser_tool", "data", "ANNOY_tree.ann")

navec = Navec.load(NAVEC_PATH)
vocabulary = navec.vocab.words
word_to_index = dict()
for i, word in enumerate(vocabulary):
    word_to_index[word] = i

In [45]:
# LOAD JSON WITH AVERAGE VALS
with open(f'../../files2big/beforeandafter768D0', 'r', encoding='utf-8') as f:
    jsonVecs = json.load(f)

In [46]:
word2vec = dict()
for v in jsonVecs:
    if 'Average' in v['word']:
        word2vec[v['word'].replace('_Average','')] = v['vector']

In [67]:
nlp(word)[0].pos_

'NOUN'

In [61]:
vecDict = dict()
for direction in posDict:
    vecDict.setdefault(direction, dict())
    for pos in posDict[direction]:
        vecDict[direction].setdefault(pos, [])
        keywords = [d['keyword'] for d in posDict[direction][pos]]
        for word in tqdm(keywords):
            try:
                navector = navec[word].tolist()
            except:
                print('not in navec dictionary')
                navector = np.zeros(300).tolist()
            vecDict[direction][pos].append({
                "word" : word,
                "translatedword" : ts.google(word),
                "bertvector" : word2vec[word],
                'navecvector' : navector,
                'countryInfo' : countryBreakdowns[word]
            })

100%|█████████████████████████████████████████████| 2/2 [00:02<00:00,  1.03s/it]


In [ ]:
for 

In [173]:
with open(f'../significantPCA/sigMeanDiffVecsTranslated4.json', 'w', encoding='utf-8') as f:
    json.dump(vecDict, f, ensure_ascii=False, indent=4)

In [176]:
with open(f'../significantPCA/sigMeanDiffVecsTranslated4.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [178]:
tdata = dict()
for iod in data:
    for pos in data[iod]:
        for word in data[iod][pos]:
            tdata[word['word']] = word['translatedword']

In [180]:
with open(f'../sigFreqTables//translations.json', 'w', encoding='utf-8') as f:
    json.dump(tdata, f, ensure_ascii=False, indent=4)

### Create word category datasets

In [182]:
with open('untitled.txt','r') as f:
    txt = f.read()

In [186]:
sDict = dict()
for subject in txt.split('\n'):
    split = subject.split(' - ')
    sDict[split[0]] = [d.strip() for d in split[1].split(',') if d.strip() != '']

In [188]:
with open(f'../averagePCA/categories.json', 'w', encoding='utf-8') as f:
    json.dump(sDict, f, ensure_ascii=False, indent=4)

## Synsets

In [1]:
!ruwordnet download

downloading a ruwordnet model from https://github.com/avidale/python-ruwordnet/releases/download/0.0.4/ruwordnet-2021.db


In [2]:
from ruwordnet import RuWordNet
wn = RuWordNet()

In [11]:
lemma = 'мир'
for sense in wn.get_senses(lemma):
    print(sense.synset.title.lower())

вселенная, мироздание, мир
сообщество (совокупность людей)
мир, отсутствие конфликтов
мир, отсутствие войны


In [12]:
lemmaCts = dict()
totalPoemCts = dict()
scipyArray = dict()
sense2lemmas = dict()

for r in tqdm(records):
    p = r['Before or after']
    totalPoemCts.setdefault(p, 0)
    lemmasFound = set()
    totalPoemCts[p] += 1
    for t in r['doc']:
        if t.lemma_ not in lemmasFound:
            for sense in wn.get_senses(t.lemma_):
                title = sense.synset.title.lower()
                sense2lemmas.setdefault(sense, set())
                sense2lemmas[sense].add(t.lemma_)
                lemmaCts.setdefault(title, {'Before' : 0, 'After' : 0})
                lemmaCts[title][p] += 1
            lemmasFound.add(t.lemma_)
        else:
            continue
lemmaFreqs = dict()
for word in lemmaCts:
    lemmaFreqs.setdefault(word, {'Before' : 0, 'After' : 0})
    for p in lemmaCts[word]:
        lemmaFreqs[word][p] = lemmaCts[word][p]/totalPoemCts[p]
        lemmaFreqs[word][p+'Ct'] = lemmaCts[word][p]

NameError: name 'tqdm' is not defined

In [ ]:
    # check if title already added
    title = synset.title
    # change synset label to lemma label if only one lemma mentioned
    if len(synsetTokens[synset]) == 1:
        title = list(synsetTokens[synset].keys())[0]
        
    # if title already added, skip entry, else continue
    if title in titleAddedAlready:
        print('OLD:',title, synset.title)
        print('NEW:',title, lemma2synset[title])
        continue
    else:
        lemma2synset[title] = synset.title
        titleAddedAlready.add(title)

#### Language Detection

In [2]:
# !pip3 install spacy-language-detection

In [3]:
import spacy
from spacy.language import Language

from spacy_language_detection import LanguageDetector


def get_lang_detector(nlp, name):
    return LanguageDetector(seed=42)  # We use the seed 42


In [4]:

nlp_model = spacy.load("ru_core_news_lg")
Language.factory("language_detector", func=get_lang_detector)
nlp_model.add_pipe('language_detector', last=True)

# Sentence level language detection
text = "This is English text. Er lebt mit seinen Eltern und seiner Schwester in Berlin. Yo me divierto todos los días en el parque. Je m'appelle Angélica Summer, j'ai 12 ans et je suis canadienne."
doc = nlp_model(text)
for i, sent in enumerate(doc.sents):
    print(sent, sent._.language)

This is English text. {'language': 'en', 'score': 0.9999987929307774}
Er lebt mit seinen Eltern und seiner Schwester in Berlin. {'language': 'de', 'score': 0.999996045846908}
Yo me divierto todos los días en el parque. {'language': 'es', 'score': 0.9999960751128256}
Je m'appelle Angélica Summer, j'ai 12 ans et je suis canadienne. {'language': 'fr', 'score': 0.9999960488878062}


In [ ]:
from tqdm import tqdm 
random.shuffle(records)
for rec in tqdm(records):
    rec['Doc'] = nlp_model(rec['Text'])

100%|███████████████████████████████████████| 3252/3252 [05:13<00:00, 10.36it/s]


In [105]:
langDict = dict()
for rec in tqdm(records):
    for sent in rec['Doc'].sents:
        langDict.setdefault(sent._.language['language'], [])
        langDict[sent._.language['language']].append((sent, rec))

100%|███████████████████████████████████████| 3252/3252 [07:47<00:00,  6.96it/s]


In [106]:
langDict.keys()

dict_keys(['ru', 'UNKNOWN', 'bg', 'mk', 'uk', 'en', 'et', 'hr', 'tl', 'ro', 'lt', 'de', 'id', 'ca', 'sk'])

In [146]:
lg = 'sk'
print(len(langDict[lg]))
[print('Unique index:',d[1]['UniqueIndex'],'\n', d[0],'\n--') for d in random.sample(langDict[lg],1)]

1
Unique index: 1652 
 Како добро живети…
…Ako žiť dobre…
…Kako dobro živeti. 
--


[None]

In [151]:
# 523
r = [r for r in records if r['UniqueIndex'] == 3239][0]

In [157]:
r['Source'], r['Author'], r['Date posted']

('essentialpoetry', 'Линор Горалик', datetime.datetime(2018, 5, 9, 0, 0))

In [153]:
print(r['Text'])

Вот красным лесом красная лиса, – 
а он лежит, смешавшись с автоматом, 
в осеннем красном буром чернозёме, 
неглубоко, – 
и вот лиса несётся, 
пересекая сердце, горло, сердце, – 
подскакивает, лапой влезши в душу, 
отряхивает лапу, мчится дальше, – 
И он кричит распавшейся гортанью: 
 
КАКОГО ХУЯ, ГОСПОДИ, – ЗА ЧТО?! 
 
Я не успел – я инвалид по зренью, – 
я не успел, – они меня в апреле, 
когда уже исход и всё понятно, 
когда таких, как я, – едва одетых, 
полуслепых, хромых или безусых, – 
от киндер, кирхе, запаха из кюхен, – 
в зелёный их апрельский красный лес, 
где я от крови ничего не видел, 
и красный зверь, и горло, сердце, горло – 
а я ни разу даже не пальнул, 
я не успел – 
какого хуя, Боже?! 
 
ТАК ДАЙ МНЕ, ДАЙ МНЕ, ДАЙ МНЕ ЧТО-НИБУДЬ!!! 
 
И тут лиса упала и лежит.


In [133]:
# other

In [2]:
total = 0
for rec in records:
    total += len(str(rec['Text']).split(' '))
# 345043
total

342235

In [5]:
df['Source'].value_counts()

essentialpoetry      945
No War Poetry        724
metajournal          662
ROAR V3              350
Facebook             253
ROAR V2              234
Personal Telegram     54
Name: Source, dtype: int64

In [4]:
df['Before or after'].value_counts()

Before    1661
After     1561
Name: Before or after, dtype: int64

### Remove duplicates from corpus

In [222]:
print(random.choice(records)['Text'][:200])

Развивая Бродского
1
Я хотел бы жить, Иосиф, как можно дольше, пока
ритмично дыханье и тянется на бумаге строка,
жизнь - гостиный двор, мы временные постояльцы,
и какое мне дело до того городка,
где и


In [223]:
longStringCts = dict()
string2Recs = dict()
string2RecIdx = dict()

windowSize = 100
for i, rec in tqdm(enumerate(records)):
    text = rec['Text']
    rec['UniqueIndex'] = i
    if isinstance(text, str):
        for j in range(len(text)-windowSize):
            windowText = text[j:j+windowSize]
            longStringCts.setdefault(windowText, 0)
            longStringCts[windowText] += 1
            string2Recs.setdefault(windowText, [])
            string2Recs[windowText].append(rec)
            # string2RecIdx.setdefault(windowText, set())
            # string2RecIdx[windowText].add(i)

3502it [00:08, 406.86it/s] 


In [224]:
relevantCts = dict()
for s in tqdm(longStringCts):
    if longStringCts[s] > 1:
        relevantCts[s] = longStringCts[s]

100%|████████████████████████████| 2284719/2284719 [00:01<00:00, 1186974.80it/s]


In [225]:
overlappingRecs = []
for s in tqdm(relevantCts):
    shareString = string2Recs[s]
    if shareString not in overlappingRecs:
        # ids = [r['UniqueIndex'] for r in shareString]
        # if set(ids) 
        overlappingRecs.append(shareString)
len(overlappingRecs)

100%|█████████████████████████████████| 64074/64074 [00:00<00:00, 133374.94it/s]


228

In [143]:
import datetime

In [226]:
def decideRec(recList):
    # if just one, return rec
    if len(recList) == 1:
        return recList[0:]
    # convert to datetimes
    for r in recList:
        if not isinstance(r['Date posted'], datetime.datetime):
            # essentialpoetry      1023
            # No War Poetry         793
            # metajournal           713
            # ROAR V3               357
            # Facebook              281
            # ROAR V2               269
            # Personal Telegram      66
            if r['Source'] == 'ROAR V2':
                r['Date posted'] = datetime.datetime(2022, 6, 24, 0, 0)
            elif r['Source'] == 'ROAR V3':
                r['Date posted'] = datetime.datetime(2022, 8, 24, 0, 0)
            else:
                r['Date posted'] = datetime.datetime(2022,12,17,0,0)
    # compare date
    sortedByDate = []
    for i, rec in enumerate(recList):
        sortedByDate.append((rec['Date posted'], i))
    sortedByDate = sorted(sortedByDate)
    # check types
    if isinstance(sortedByDate[0], datetime.datetime) and isinstance(sortedByDate[1], datetime.datetime):
        if sortedByDate[0] != sortedByDate[1]:
            return recList[sortedByDate[0][1]]
    else:
        idxs = [s[1] for s in sortedByDate[1:]]
        return [recList[j] for j in idxs]
    
    # compare length of text
    sortedByLen = []
    for i, rec in enumerate(recList):
        sortedByLen.append((len(rec['Text']), i))
    sortedByLen = sorted(sortedByLen)
    idxs = [s[1] for s in sortedByLen[1:]]
    return [recList[j] for j in idxs]

In [229]:
records[162]

{'Text': '\nРевёт жена опальная,\nв правах поражена.\nИдёт война двуспальная,\nгражданская война.\nПомыты кухня, ванная\nслезами в три ручья.\nИдёт война диванная,\nтрещит по швам семья.\n+\nБудущему малышу,\nптицам небесным пища?\nСижу в уголке, пишу\nугольком пепелища\nродного. Роднее нет.\nМай. Четвёртое мая.\nВ окне кабинета свет:\nне выключила, убегая.\n+\nВсе страны, кроме одной.\nЧто же мы так бездомны?\nВстречаемся, мой родной,\nоколо пятой колонны\nбольшого театра войны —\nв Эривани, в Тифлисе,\nкак рыбе зонтик нужны\nмировой закулисе.\n+\nОт грохота, скрежета, лязга\nсжимается матка.\nКачает пустую коляску.\nЦе хлопчик? Дiвчатко.\nПоёт колыбельную дочке.\nПечаль непроглядна.\nВойна. Молока на сорочке\nрасплывающиеся пятна.\n+\nСвидетельств — терабайт,\nкак мой народ ушёл\nв искусственный офсайд.\nНо засчитали гол\nи торжествует зло,\nи стадион орёт.\nМне очень повезло\nуспеть на самолёт.\n+\nМеждународный язык —\nмладенческий лепет.\nСпит у груди призывник\nГолгофы. Колеблет\

In [230]:
counts = dict()
for recList in overlappingRecs:
    counts.setdefault(len(recList),0)
    counts[len(recList)] += 1

In [231]:
# create new list using only the earliest overlapping rec
repeatIdxs = set()
for recList in overlappingRecs:
    badRecs = decideRec(recList)
    badIdxs = [b['UniqueIndex'] for b in badRecs]
    for b in badIdxs:
        repeatIdxs.add(b)

In [232]:
cleanRecList = []
j = 0
for i, rec in enumerate(records):
    if rec['UniqueIndex'] not in repeatIdxs:
        rec['UniqueIndex'] = j
        cleanRecList.append(rec)
        j += 1
len(cleanRecList)

3289

In [234]:
cleanRecList[1000]

{'Text': '***\n \n– В Москве – невкусная клубника, бери водку!\n \nВ вытянутой трубке сна так тихо,\n \nЧто можно услышать\n \nНеуловимую чёрную лодку.\n \nСпрашиваю:\n \n– Это каяк?\n \nПтица с ветки:\n \n– Каяк, каяк. Что с тобою не так?\n \n– Со мною не так носовой платок,\n \nКаждое слово.\n \nНеба бесформенный носорог\n \nОживает в половине шестого.\n \nНаблюдаю за ним из окна.\n \nУ меня проблема только одна:\n \n«Скендербеу», «Хабнарфьордюр» и «Нымме Калью» проиграли свои матчи.\n \nИ три любимых, но непростых занятья:\n \nСмотреть на осину,\nСмотреть на берёзу,\nСмотреть на тополь.\n \nМошка умерла у меня на глазах,\nА что пережил ты?\n \n \n(интернет-журнал ”Literratura”)\n \n#выбор_Максима_Дрёмова',
 'Author': 'Денис Крюков',
 'Before or after': 'Before',
 'Source': 'metajournal',
 'Date posted': datetime.datetime(2019, 11, 14, 0, 0),
 'UniqueIndex': 1000}

In [233]:
cleanDf = pd.DataFrame.from_records(cleanRecList)

In [235]:
cleanDf.to_excel('../Excel_files/Full_Poem_Dataset_12-17.xlsx')

## AUTHOR DATA

In [7]:
authorInfo = pd.read_excel('../Excel_files/Thesis Authors.xlsx')
authorInfo.head()

,Number of Poems,Author,City,Country,Identification,Notes,Link to bio
0,3,Виген Аракелян,Razdan,Armenia,NaN,NaN,https://polutona.ru/?show=arakelian
1,1,Герман Лукомников,Baku,Azerbaijan,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%9B%D1%83%D0%...
2,1,Александр Иличевский,Sumgait,Azerbaijan,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%98%D0%BB%D0%...
3,7,Леонид Шваб,Babruysk,Belarus,NaN,NaN,http://www.litkarta.ru/world/israel/persons/sh...
4,6,Ирина Валерина,Babruysk,Belarus,NaN,NaN,NaN


In [9]:
authorInfo['Country'].value_counts()

Russia          370
Ukraine          86
Kazakhstan       26
Belarus          16
Uzbekistan        6
Moldova           5
Latvia            5
Estonia           4
Kyrgyzstan        2
Azerbaijan        2
Lithuania         1
Iran              1
Georgia           1
Tajikistan        1
Turkmenistan      1
USA               1
Armenia           1
Name: Country, dtype: int64

In [ ]:
value_counts